In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
from tqdm.notebook import tqdm

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, TQDMProgressBar


import matplotlib.pyplot as plt
%matplotlib qt

DATASET_PATH = '/home/pk/Downloads/PASCAL_VOC/'

CHECKPOINT_PATH = '../saved_models/yolo/'

GPU_NUMBER = 1

device = torch.device("cuda:" + str(GPU_NUMBER)) if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:1


In [3]:
from networks import NETWORK_CONFIG, ANCHORS, YOLOv3
from datasets import YOLODataset
from losses import YoloLoss
from transformations import YoloTrainTransformations

In [6]:
train_csv_file = DATASET_PATH + '100examples.csv'
test_csv_file = DATASET_PATH + '100examples.csv'
img_dir = DATASET_PATH + 'images/'
label_dir = DATASET_PATH + 'labels/'
anchors = ANCHORS

BATCH_SIZE = 16
IMAGE_SIZE=416
S = [13, 26, 52]
NUM_CLASSES = 20

SCALED_ANCHORS = (
    torch.tensor(ANCHORS)
    * torch.tensor(S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
)


train_data = YOLODataset(train_csv_file, img_dir, label_dir, 
                                        anchors,
                                        image_size=IMAGE_SIZE,
                                        S=S,
                                        C=NUM_CLASSES,
                                       transform=YoloTrainTransformations())

test_data = YOLODataset(test_csv_file, img_dir, label_dir, 
                                       anchors,
                                       image_size=IMAGE_SIZE,
                                       S=S,
                                       C=NUM_CLASSES,
                                       transform=YoloTrainTransformations())

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=4)


In [7]:
input_data = next(iter(train_loader))

##### Input and output data shapes

In [8]:
images, targets = input_data

In [9]:
images.shape

torch.Size([16, 3, 416, 416])

In [10]:
targets[0].shape, targets[1].shape, targets[2].shape

(torch.Size([16, 3, 13, 13, 6]),
 torch.Size([16, 3, 26, 26, 6]),
 torch.Size([16, 3, 52, 52, 6]))

In [11]:
net = YOLOv3(c_in=3, config=NETWORK_CONFIG, num_classes=20)
print(f"Net input shapes: {images.shape}")
# output generated is 3 targets for each scale
out_net = net(images)
print("Net output shapes:")
for tensor in out_net:
    print(tensor.shape)

Net input shapes: torch.Size([16, 3, 416, 416])
Net output shapes:
torch.Size([16, 3, 13, 13, 25])
torch.Size([16, 3, 26, 26, 25])
torch.Size([16, 3, 52, 52, 25])


#### Loss per batch 

Loss function needs three things, predictions, targets and anchors

In [12]:
loss = YoloLoss()

In [13]:
out_net[0].shape, out_net[1].shape, out_net[2].shape

(torch.Size([16, 3, 13, 13, 25]),
 torch.Size([16, 3, 26, 26, 25]),
 torch.Size([16, 3, 52, 52, 25]))

In [14]:
loss(out_net[0], targets[0], SCALED_ANCHORS[0])

tensor(13.2959, grad_fn=<AddBackward0>)

### Trainer function

In [15]:
from lightning_modules import YOLOTrainingModule

In [16]:
def train_model(model_name, max_epochs=20, save_name=None, retrain=True, **kwargs):
    """
    Inputs:
        model_name - Name of the model to run
        save_name (optional) - This name will be used in checkpoint and logging directory
    """
    if save_name is None:
        save_name = model_name

    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, save_name),
                        gpus=[1],
                        max_epochs=max_epochs,
                        callbacks=[ModelCheckpoint(save_weights_only=True, mode="min",
                                                   monitor="train_loss"),
                                   LearningRateMonitor("epoch"),
                                   TQDMProgressBar(refresh_rate=10)],
                        log_every_n_steps=10
                        )
    trainer.logger._log_graph = False
    trainer.logger._default_hp_metric = None
    
    # pretrained, then load and skip it
    pretrained_filename = os.path.join(CHECKPOINT_PATH, save_name+".ckpt")
    if os.path.isfile(pretrained_filename) and not re_train:
        print(f"Found pretrained model at {pretrained_filename}, loading ...")
        model = YOLOTrainingModule.load_from_checkpoint(pretrained_filename)
    else:
        #pl.seed_everything(42)
        model = YOLOTrainingModule(model_name=model_name, **kwargs)
        trainer.fit(model, train_loader, test_loader)
        model = YOLOTrainingModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    
    # test for accuracy/ whatever is logged. we only log losses for now.
    train_result = trainer.test(model, dataloaders=train_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
    
    result = {"train": train_result[0]["test_loss"], "test": test_result[0]["test_loss"]}
    
    return model, result

#### Train Run

In [17]:
max_epochs = 1000
model, result = train_model(
    "YOLOv3",
    max_epochs=max_epochs,
    model_hparams= {
        "c_in": 3,
        "config": NETWORK_CONFIG,
        "num_classes": NUM_CLASSES,
    },
    loss_name="YoloLoss",
    loss_hparams={},
    optimizer_name="Adam",
    optimizer_hparams={"lr": 3e-4, "weight_decay": 1e-4},
    scheduler_hparams={
        
    },
    device=device,
    scaled_anchors=SCALED_ANCHORS
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type     | Params
-----------------------------------------
0 | model       | YOLOv3   | 61.6 M
1 | loss_module | YoloLoss | 0     
-----------------------------------------
61.6 M    Trainable params
0         Non-trainable params
61.6 M    Total params
246.506   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/pk/anaconda3/envs/simulator/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:412: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/pk/anaconda3/envs/simulator/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:633: UserWarning: Your `test_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

In [18]:
result

{'train': 2.1315364837646484, 'test': 2.270245313644409}

In [19]:
model.state_dict()

OrderedDict([('model.layers.0.conv.weight',
              tensor([[[[-1.8516e-01, -8.5758e-02,  7.0946e-02],
                        [ 1.5649e-02, -1.7461e-01,  4.0539e-02],
                        [ 4.0744e-02,  1.1502e-01, -5.1041e-02]],
              
                       [[ 1.0634e-01,  5.3764e-02,  2.3977e-02],
                        [-1.0945e-01,  6.2339e-03,  6.6861e-02],
                        [ 1.7802e-01, -1.1137e-01,  7.3500e-02]],
              
                       [[ 6.7440e-02,  5.6948e-03, -7.9384e-02],
                        [-1.2063e-02, -8.3775e-02, -4.5712e-03],
                        [ 1.0094e-01,  1.5454e-01, -2.0971e-03]]],
              
              
                      [[[ 1.2037e-03, -1.5644e-01, -1.5223e-01],
                        [ 9.6504e-02,  2.1760e-02, -1.6647e-01],
                        [ 5.7352e-02, -8.9981e-02, -6.5278e-02]],
              
                       [[-5.3368e-04, -7.4490e-02,  1.4437e-01],
                        [ 7.066

#### Let's load from checkpoint and re-save

```
chk_path = '/home/pk/Documents/narsil2/saved_models/yolo/YOLOv3/lightning_logs/version_0/checkpoints/epoch=396-step=2778.ckpt'

model_from_chk = YOLOTrainingModule.load_from_checkpoint(chk_path)
model_dict = model_from_chk.state_dict()
MODEL_SAVE_PATH = '../saved_models/pth_files/yolo_100samples.pth' torch.save(model_from_chk.state_dict(), MODEL_SAVE_PATH)

```

# 

### Testing model predictions